In [1]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
import xarray as xr
import os; import time
import pickle
import h5py

In [2]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Variable_Calculation/EntrainmentCalculation'

In [3]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data,parcel] = GetData(dataDirectory, parcelDirectory)

In [4]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'

In [ ]:
#########################################

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
############################################################
#PLOTTING
plotting=False #KEEP FALSE IF JOB ARRAY IS RUNNING
plotting=True

In [ ]:
#READING BACK IN
# import pickle
# dir2 = dir + f'Project_Algorithms/Domain_Profiles/'
# input_file = dir2 + f'MeanLFC_{res}_{t_res}_{Np_str}.pkl'

# with open(input_file, 'rb') as f:
#     MeanLFC = pickle.load(f)
# print(MeanLFC)

def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC}\n")


def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {cloudbase}\n")

In [ ]:
if plotting==True:
    #constants
    Cp=1004 #Jkg-1K-1
    Cv=717 #Jkg-1K-1
    Rd=Cp-Cv #Jkg-1K-1
    eps=0.608
    
    Lx=(data['xf'][-1].item()-data['xf'][0].item())*1000 #x length (m)
    Ly=(data['yf'][-1].item()-data['yf'][0].item())*1000 #y length (m)
    Np=len(parcel['xh']) #number of lagrangian parcles
    dt=(data['time'][1]-data['time'][0]).item()/1e9 #sec
    dx=(data['xf'][1].item()-data['xf'][0].item())*1e3 #meters
    dy=(data['yf'][1].item()-data['yf'][0].item())*1e3 #meters
    xs=data['xf'].values*1000
    ys=data['yf'].values*1000
    zs=data['zf'].values*1000
    
    def zf(z):
        k=z #z is the # level of z
        out=data['zf'].values[k]*1000
        
        return out
    # def rho(x,y,z,t):
    #     p=data['prs'].isel(xh=x,yh=y,zh=z,time=t).item()
    #     p0=101325 #Pa
    #     theta=data['th'].isel(xh=x,yh=y,zh=z,time=t).item()
    #     T=theta*(p/p0)**(Rd/Cp)
    #     qv=data['qv'].isel(xh=x,yh=y,zh=z,time=t).item()
    #     # Tv=T*(1+eps*qv)
    #     Tv=T*(eps+qv)/(eps*(1+qv))
    #     rho = p/(Rd*Tv)
    #     out=rho
    #     return out
    
    def rho(x,y,z,rho_data_t):
        out=rho_data_t[z,y,x]
        return out
    def m(t):
        rho_data_t=data['rho'].isel(time=t).data
        
        m=0
        #triple sum
        for k in range(len(data['zh'])):
            dz=(zf(k+1)-zf(k))
            for j in range(len(data['yh'])):
                for i in range(len(data['xh'])):
                    rho_out=rho(i,j,k,rho_data_t)
                    m+=rho_out*dz
                    
        #triple sum
        out=m*dx*dy/Np
        return out


In [ ]:
#SOME CALCULATIONS (TESTING)
# # (Lx*Ly*(10**4))/1e6 #1M parcels ==> 1 billion kg/parcel
# # (Lx*Ly*(10**4))/50e6 #50M parcels ==> 20 million kg/parcel
# # (Lx*Ly*(10**4))/100e6 #100M parcels ==> 10 million kg/parcel

# # 1e5 kg | 9.1125 e4 m^3
# # x   kg | 1000*1000*62 = 6.2e7 m^3
# (1000*1000*62)*(1e5/(9.1125e4))# ==> 68038408 ==> should have 68M kg in the bottom most layer 

# 68038408/19729158# (expected/calculated mass) ==> should have 3.5 parcels in each grid box on the bottom most layer
# #we have 369e3 parcels on bottom layer ==> 369e3/(Nx*Ny) = 369e3/102400 = 3.6 parcels per layer!

In [ ]:
if plotting==True:
    #Calculate Mass Constant
    # calculate='single_time'
    # calculate=True
    calculate=False
    
    if calculate==True:
        Nt=len(data['time'])
        m_arr=np.zeros((Nt))
        for t in np.arange(Nt):
            if np.mod(t,25)==0: print(t)
            m_arr[t]=m(t)
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
        np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
    elif calculate=='single_time':
        Nt=len(data['time'])
        m_arr=np.zeros((Nt))
    
        t=0 #len(data['time'])//2 #Pick some middle time
        m_300=m(t)
        for t in np.arange(Nt):
            m_arr[t]=m_300 #UNCOMMENT FOR FULL CALCULATION
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
        np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
    else:
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
        m_arr = np.load(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy')
    
    # # TESTING
    # lst=[]
    # for t in np.arange(133):
    #     lst.append(m_arr[t])
    
    # plt.plot(lst)
    # (np.max(lst)-np.min(lst))*100/np.mean(lst)

In [ ]:
if plotting==True:
    PROCESSING=False
    PROCESSING=True
    
    dir3=dir+'Project_Algorithms/Entrainment/OUTPUT/'
    if PROCESSING==False:
        open_file=dir3+f'2D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
    if PROCESSING==True:
        open_file=dir3+f'2D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(open_file, "r") as h5f:
        profile_array_e_g = h5f["profile_array_e_g"][:]
        profile_array_e_c = h5f["profile_array_e_c"][:]
        profile_array_d_g = h5f["profile_array_d_g"][:]
        profile_array_d_c = h5f["profile_array_d_c"][:]

In [ ]:
if plotting==True:
    def apply_constant(profile_array,apply):
        if apply==True:
            Nt=profile_array.shape[0]
            Nz=profile_array.shape[1]
        
            profile_array/=(Lx*Ly*dt)
            for t in np.arange(Nt):
                profile_array[t]*=m_arr[t]
            for z in np.arange(Nz):
                dz=zf(z+1)-zf(z)
                profile_array[:,z]/=dz
        return profile_array
    #APPLY CONSTANTS TO ENTRAINMENT VALUE
    ##################################################
    profile_array_e_g=apply_constant(profile_array_e_g,apply=True)
    profile_array_e_c=apply_constant(profile_array_e_c,apply=True)
    profile_array_d_g=-apply_constant(profile_array_d_g,apply=True)
    profile_array_d_c=-apply_constant(profile_array_d_c,apply=True)
    ##################################################

In [ ]:
if plotting==True:
    PROCESSING=False
    PROCESSING=True
    
    dir3=dir+'Project_Algorithms/Entrainment/OUTPUT/'
    # if PROCESSING==False:
    #     open_file=dir3+f'2D_entrainmentdetrainment_combined_profiles_{res}_{t_res}_{Np_str}.h5'
    # if PROCESSING==True:
    #     open_file=dir3+f'2D_entrainmentdetrainment_combined_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
    if PROCESSING==False:
        open_file=dir3+f'2D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
    if PROCESSING==True:
        open_file=dir3+f'2D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(open_file, "r") as h5f:
        profile_array_c_to_g_E = h5f["profile_array_c_to_g_E"][:]
        profile_array_g_to_c_E = h5f["profile_array_g_to_c_E"][:]
        profile_array_c_to_g_D = h5f["profile_array_c_to_g_D"][:]
        profile_array_g_to_c_D = h5f["profile_array_g_to_c_D"][:]

In [ ]:
if plotting==True:
    def apply_constant(profile_array,apply):
        if apply==True:
            Nt=profile_array.shape[0]
            Nz=profile_array.shape[1]
        
            profile_array/=(Lx*Ly*dt)
            for t in np.arange(Nt):
                profile_array[t]*=m_arr[t]
            for z in np.arange(Nz):
                dz=zf(z+1)-zf(z)
                profile_array[:,z]/=dz
        return profile_array
    #APPLY CONSTANTS TO ENTRAINMENT VALUE
    ##################################################
    profile_array_c_to_g_E=apply_constant(profile_array_c_to_g_E,apply=True)
    profile_array_g_to_c_E=apply_constant(profile_array_g_to_c_E,apply=True)
    profile_array_c_to_g_D=apply_constant(profile_array_c_to_g_D,apply=True)
    profile_array_g_to_c_D=apply_constant(profile_array_g_to_c_D,apply=True)
    ##################################################

In [ ]:
def GetData(type):
    if type=='general':
        profile_array_e=profile_array_e_g
        profile_array_d=profile_array_d_g
        profile_array_net=profile_array_e-profile_array_d
    if type=='cloudy':
        profile_array_e=profile_array_e_c
        profile_array_d=profile_array_d_c
        profile_array_net=profile_array_e-profile_array_d

    return profile_array_e,profile_array_d,profile_array_net

In [ ]:
def plot_mean_entrainment(ax, profile_array_e, profile_array_d, title, linestyle='solid'):
    zh=data['zh'].data

    # Compute mean profiles
    e = np.mean(profile_array_e, axis=0)
    d = np.mean(profile_array_d, axis=0)
    net = np.mean(profile_array_e - profile_array_d, axis=0)

    # Plot
    ax.plot(e, zh, linestyle=linestyle, color='blue', label='Entrainment')
    ax.plot(d, zh, linestyle=linestyle, color='red', label='Detrainment')
    ax.plot(net, zh, linestyle=linestyle, color='black', label='Net Entrainment')
    ax.axvline(0, color='gray', linestyle='dashed',zorder=-10)

    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed', lw=1.2)

    ax.grid(True)
    ax.set_title(f"{title}",fontsize=10.5)
    ax.set_xlabel(r"($kg\ m^{-3}\ s^{-1}$)")  
    ax.set_ylabel('z (km)')
    ax.set_ylim(bottom=0)
    ax.legend(loc='upper right')
    ax.set_ylim(0,20)

    #FIXED TICKS
    SnapLimitsToTicks([ax], dim='x')

    
    # Format x-axis in scientific notation
    apply_scientific_notation([ax],decimals=2)

In [ ]:
def SaveFigure(fig,filename):
    save_dir = f"PLOTS/{res}_{t_res}_{Np_str}"
    os.makedirs(save_dir, exist_ok=True)
    fig.savefig(f"{save_dir}/{filename}_{res}_{t_res}_{Np_str}.jpg", dpi=300, bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter
zh=data['zh'].data
def Plot_EntrainmentDetrainmentRows(data, array_triplets, type_labels, plotting=True):
    if not plotting:
        return
        
    cmap1 = plt.cm.viridis
    cmap2 = plt.cm.seismic 
    n_levels = 29
    z_lim=20

    num_rows = len(array_triplets)

    fig = plt.figure(figsize=(18, 4 * num_rows))
    gs = GridSpec(num_rows, 4, figure=fig)
    all_ax1, all_ax2, all_ax3 = [], [], []
    
    for row_idx, ((profile_array_e, profile_array_d, profile_array_net), type_label) in enumerate(zip(array_triplets, type_labels)):
        vmax_shared = np.max([np.max(profile_array_e), np.max(profile_array_d)])
        norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
        norm_shared = None

        y = data['zh'].data
        x = np.arange(profile_array_e.shape[0])

        mins=(data['time'].data[1].astype(int)/1e9/60)
        plot_kwargs = {
            'PlotData': None,
            'xTickLabels': x, 'yTickLabels': y,
            'contour_type': 'line',#'fill',
            'num_xticks': 11, 'round_xticks': None, 'xTickInterval': 60/mins,
            'num_yticks': 15, 'round_yticks': 2, 'yTickInterval': None,
            'add_colorbar': True, 'fig': fig, 'levels': 29, 'colorbar_label_rotation': 0, 'colorbar_label': None,
            'xlabel': "t (hours)", 'ylabel': "z (km)",
            'solid_contour_labels': False, 'solid_contour_round': None,
            'xtick_rotation': 0, 'ytick_rotation': 0, 'cbar_rotation': 0,
            'save_path': None, 'save_dpi': 300,
            'colorbar_kwargs': {
                'extend': 'both'
            },
            'norm': norm_shared
        }

        ax1 = fig.add_subplot(gs[row_idx, 0])
        ax2 = fig.add_subplot(gs[row_idx, 1])
        ax3 = fig.add_subplot(gs[row_idx, 2])

        all_ax1.append(ax1)
        all_ax2.append(ax2)
        all_ax3.append(ax3)

        plot_kwargs1 = plot_kwargs.copy()
        plot_kwargs1['PlotData'] = profile_array_e.copy().T
        plot_kwargs1['cmap'] = cmap1
        [contour1, cbar1] = UltimateContourPlot(ax1, **plot_kwargs1)
        ax1.set_ylim(0, z_lim)
        ax1.set_title(f'Entrainment {type_label}',fontsize=8)

        plot_kwargs2 = plot_kwargs.copy()
        plot_kwargs2['PlotData'] = profile_array_d.copy().T
        plot_kwargs2['cmap'] = cmap1
        [contour2, cbar2] = UltimateContourPlot(ax2, **plot_kwargs2)
        ax2.set_ylim(0, z_lim)
        ax2.set_title(f'Detrainment {type_label}',fontsize=8)

        plot_data3 = profile_array_net.copy().T
        vmin = -np.max(abs(profile_array_net)) / 2
        vmax = +np.max(abs(profile_array_net))
        levels = np.linspace(vmin, vmax, n_levels)
        norm = mcolors.BoundaryNorm(boundaries=levels, ncolors=256)

        plot_kwargs3 = plot_kwargs.copy()
        plot_kwargs3['PlotData'] = plot_data3
        plot_kwargs3['cmap'] = cmap2
        plot_kwargs3['norm'] = norm
        plot_kwargs3['levels'] = levels
        [contour3, cbar3] = UltimateContourPlot(ax3, **plot_kwargs3)
        ax3.set_ylim(0, 19)
        ax3.set_title(f'Net Entrainment {type_label}',fontsize=8)
        # fig.suptitle(f"{type_labels[row_idx][0].upper() + type_labels[row_idx][1:]} Updraft Entrainment/Detrainment")

        def apply_scientific_notation_colorbar(cbars):
            from matplotlib.ticker import ScalarFormatter
            formatter = ScalarFormatter(useMathText=True)
            formatter.set_powerlimits((-2, 2))
            for cbar in cbars:
                cbar.formatter = formatter
                cbar.update_ticks()

        apply_scientific_notation_colorbar([cbar1, cbar2, cbar3])

        #FIXING CONTOUR COLORBAR LINES THICKNESS
        for cbar in [cbar1, cbar2, cbar3]:
            for cbar_line in cbar.ax.collections:
                cbar_line.set_linewidth(3)

        for ax in [ax1,ax2,ax3]:
            ax.axhline(cloudbase, color='purple', linestyle='dashed',lw=1.2)
            ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed',lw=1.2)
            xticks = ax.get_xticks()
            new_labels = [f"{tick*mins/(60)+6:.0f}" for tick in xticks]
            ax.set_xticklabels(new_labels)

    ticks = np.linspace(0, z_lim, num=11)
    for ax in all_ax1 + all_ax2 + all_ax3:# + [ax4, ax5]:
        ax.set_ylim([0, z_lim])
        ax.set_yticks(ticks)
        ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
        ax.margins(y=0)
            
    ax4 = fig.add_subplot(gs[0, 3])
    ax5 = fig.add_subplot(gs[1, 3])
    plot_mean_entrainment(ax4, profile_array_e1, profile_array_d1, title='Mean Vertical Profile')
    plot_mean_entrainment(ax5, profile_array_e2, profile_array_d2, title='Mean Vertical Profile')

    #SPACING
    fig.subplots_adjust(top=0.92, wspace=0.4, hspace=0.4)
    
    #SAVING
    filename=f"Entrainment_ContourPlot_DomainProfiles"
    SaveFigure(fig,filename)

In [ ]:
#LOAD DATA
type1='general';type2='cloudy'
[profile_array_e1,profile_array_d1,profile_array_net1]=GetData(type=type1)
[profile_array_e2,profile_array_d2,profile_array_net2]=GetData(type=type2)

In [ ]:
##################################################
#PLOTTING

In [ ]:
#CONTOUR PLOTS
array_triplets = [
    (profile_array_e1, profile_array_d1, profile_array_net1),
    (profile_array_e2, profile_array_d2, profile_array_net2)
]
type_labels = ['(General Updraft)', '(Cloudy Updraft)']

Plot_EntrainmentDetrainmentRows(data, array_triplets, type_labels)

In [ ]:
#COMBINED ENTRAINMENT

In [ ]:
# #ENTRAINMENT TRANSFER CONTOUR PLOTS
# if plotting==True:
#     #Plotting
#     ############################################################
#     import matplotlib.pyplot as plt
#     from matplotlib.gridspec import GridSpec
#     import numpy as np
    
#     fig = plt.figure(figsize=(10, 8))
#     gs = GridSpec(2, 2, figure=fig)
    
#     ######
#     cmap1 = plt.cm.viridis
#     cmap1 = plt.cm.seismic 
#     n_levels=30
#     ######
    
#     ######
#     vmax_shared = np.max([np.max(profile_array_c_to_g), np.max(profile_array_g_to_c)])
#     norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
#     ######
    
#     # First subplot: Entrainment
#     ########################################
#     ax1 = fig.add_subplot(gs[0, 0])
#     contour1 = ax1.contourf(profile_array_c_to_g.T, cmap=cmap1,levels=n_levels)
#     # contour1 = ax1.contourf(profile_array_e_g.T, cmap=cmap1, norm=norm_shared, levels=n_levels)
#     cbar1=fig.colorbar(contour1, ax=ax1)
#     Nz = len(data['zh'])
#     ax1.set_yticks(np.arange(Nz))
#     new_ytick_labels = np.round(data['zh'].values[:Nz], 2)
#     ax1.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
#     ax1.set_ylabel('z (km)');ax1.set_xlabel('t (timesteps)')
#     ax1.set_title('Cloudy to General')
    
#     # Second subplot: Detrainment
#     ########################################
#     ax2 = fig.add_subplot(gs[0, 1])
#     contour2 = ax2.contourf(profile_array_g_to_c.T, cmap=cmap1, levels=n_levels)
#     # contour2 = ax2.contourf(profile_array_e_c.T, cmap=cmap1, norm=norm_shared, levels=n_levels)
#     cbar2 = fig.colorbar(contour2, ax=ax2)
#     ax2.set_yticks(np.arange(Nz))
#     new_ytick_labels = np.round(data['zh'].values[:Nz], 2)
#     ax2.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
#     ax2.set_ylabel('z (km)');ax2.set_xlabel('t (timesteps)')
#     ax2.set_title('General to Cloudy')

#     ###################### FIXING Y TICKS
#     Nz = len(data['zh'])
#     step = 4  # change to 2, 5, etc. depending on how spaced you want them
#     ytick_pos = np.arange(0, Nz, step)
#     ytick_labels = np.round(data['zh'].values[ytick_pos], 2)
#     for axis in [ax1,ax2]:
#         axis.set_yticks(ytick_pos)
#         axis.set_yticklabels(ytick_labels, fontsize=8, rotation=0)

#     ###################### SCIENTIFIC NOTATION
#     apply_scientific_notation_colorbar([cbar1,cbar2])

In [ ]:
def compute_means_for_transfer(c_to_g_E, g_to_c_E, c_to_g_D, g_to_c_D,
                               profile_array_e_g, profile_array_e_c,
                               profile_array_d_g, profile_array_d_c):
    """Compute and return mean profiles for transfer rates and entrainment/detrainment arrays."""

    # Compute means of transfer rates
    c_to_g_E_mean = np.mean(c_to_g_E, axis=0)
    g_to_c_E_mean = np.mean(g_to_c_E, axis=0)
    c_to_g_D_mean = np.mean(c_to_g_D, axis=0)
    g_to_c_D_mean = np.mean(g_to_c_D, axis=0)

    # Compute means of entrainment/detrainment profiles
    mean_e_g = np.mean(profile_array_e_g, axis=0)
    mean_e_c = np.mean(profile_array_e_c, axis=0)
    mean_d_g = np.mean(profile_array_d_g, axis=0)
    mean_d_c = np.mean(profile_array_d_c, axis=0)

    return {
        'c_to_g_E_mean': c_to_g_E_mean,
        'g_to_c_E_mean': g_to_c_E_mean,
        'c_to_g_D_mean': c_to_g_D_mean,
        'g_to_c_D_mean': g_to_c_D_mean,
        'mean_e_g': mean_e_g,
        'mean_e_c': mean_e_c,
        'mean_d_g': mean_d_g,
        'mean_d_c': mean_d_c
    }

    
def plot_transfer_rate(ax, means, title):
    zh = data['zh'].data

    c_to_g_mean_E = means['c_to_g_E_mean']
    g_to_c_mean_E = means['g_to_c_E_mean']
    c_to_g_mean_D = means['c_to_g_D_mean']
    g_to_c_mean_D = means['g_to_c_D_mean']

    # ax.plot(g_to_c_mean_E, zh, color='blue', label='General → Cloudy',linestyle='solid')
    # ax.plot(c_to_g_mean_E, zh, color='red', label='Cloudy → General',linestyle='solid')
    ax.plot(g_to_c_mean_E, zh, color='blue', label='General → Cloudy (Entrainment)',linestyle='solid')
    ax.plot(c_to_g_mean_E, zh, color='red', label='Cloudy → General (Entrainment)',linestyle='solid')
    ax.plot(c_to_g_mean_D, zh, color='blue', label='Cloudy → General (Detrainment)',linestyle='dashed')
    ax.plot(g_to_c_mean_D, zh, color='red', label='General → Cloudy (Detrainment)',linestyle='dashed')
    ax.axvline(0, color='black', linewidth=1)

    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(MeanLFC / 1000, color='forestgreen', linestyle='dashed', lw=1.2)

    ax.grid(True)
    ax.set_title(f"{title}")
    ax.set_xlabel('Mass Transfer Rate')
    ax.set_xlabel(r"($kg m^{-3} s^{-1}$)")  
    ax.set_ylabel('z (km)')
    ax.set_ylim(bottom=0,top=20)
    ax.legend()
    
    SnapLimitsToTicks([ax], dim='x')
    apply_scientific_notation([ax],decimals=2)

def plot_transfer_ratio(ax, means, title):
    zh = data['zh'].data

    mean_e_g = means['mean_e_g']
    mean_e_c = means['mean_e_c']
    mean_d_g = means['mean_d_g']
    mean_d_c = means['mean_d_c']
    
    mean_c_to_g_E = means['c_to_g_E_mean']
    mean_g_to_c_E = means['g_to_c_E_mean']
    mean_c_to_g_D = means['c_to_g_D_mean']
    mean_g_to_c_D = means['g_to_c_D_mean']

    threshold = 0
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio_1 = np.where(mean_e_c > threshold, mean_g_to_c_E / mean_e_c, np.nan)
        ratio_2 = np.where(mean_d_g > threshold, mean_g_to_c_D / mean_d_g, np.nan)
        ratio_3 = np.where(mean_e_g > threshold, mean_c_to_g_E / mean_e_g, np.nan)
        ratio_4 = np.where(mean_d_c > threshold, mean_c_to_g_D / mean_d_c, np.nan)

    # print(np.nanmean(ratio_1))
    # print(np.nanmean(ratio_2))
    # print(np.nanmean(ratio_3))
    # print(np.nanmean(ratio_4))

    ax.plot(ratio_1*100, zh, color='blue', label='General → Cloudy / Cloudy Entrainment')
    ax.plot(ratio_2*100, zh, color='deepskyblue', label='General → Cloudy / General Detrainment')
    ax.plot(ratio_3*100, zh, color='red', label='Cloudy → General / General Entrainment')
    ax.plot(ratio_4*100, zh, color='orangered', label='Cloudy → General / Cloudy Detrainment')

    ax.axvline(0, color='black', linestyle='dashed', linewidth=1)
    # ax.axvline(1, color='black', linestyle='dashed', linewidth=1)
    ax.axvline(100, color='black', linestyle='dashed', linewidth=1)
    ax.axhline(cloudbase, color='purple', linestyle='dashed', lw=1.2)
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed', lw=1.2)

    ax.grid(True)
    ax.set_title(f"{title}")
    # ax.set_xlabel('Ratio')
    ax.set_xlabel('%')
    ax.set_ylabel('z (km)')
    # ax.set_xlim(-0.05, 1.05)
    ax.set_xlim(-5, 105)
    ax.set_ylim(bottom=0,top=20)
    ax.legend(fontsize=10.5-2, loc='upper right')
    # apply_scientific_notation([ax])


In [ ]:
#LOAD DATA
type1='general';type2='cloudy'
[profile_array_e1,profile_array_d1,profile_array_net1]=GetData(type=type1)
[profile_array_e2,profile_array_d2,profile_array_net2]=GetData(type=type2)

In [ ]:
# === Compute means for transfer rate and ratio plots ===
means = compute_means_for_transfer(
    c_to_g_E=profile_array_c_to_g_E,
    g_to_c_E=profile_array_g_to_c_E,
    c_to_g_D=profile_array_c_to_g_D,
    g_to_c_D=profile_array_g_to_c_D,
    profile_array_e_g=profile_array_e_g,
    profile_array_e_c=profile_array_e_c,
    profile_array_d_g=profile_array_d_g,
    profile_array_d_c=profile_array_d_c
)

# === Set up figure and subplots ===
fig = plt.figure(figsize=(18, 6))
gs = gridspec.GridSpec(1, 4, figure=fig, wspace=0.3, )


ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax3 = fig.add_subplot(gs[2])
ax4 = fig.add_subplot(gs[3])

# === Plot each panel ===
plot_mean_entrainment(ax1, profile_array_e1, profile_array_d1, title='')
plot_mean_entrainment(ax2, profile_array_e2, profile_array_d2, title='')
plot_transfer_rate(ax3, means, title='')
plot_transfer_ratio(ax4, means, title='')
fig.subplots_adjust(top=0.92, wspace=0.4, hspace=0.4)


# === Save figure ===
filename = "Combined_Entrainment_VerticalProfiles"
SaveFigure(fig, filename)

In [ ]:
#contour plot
# #Plotting
# ############################################################
# import matplotlib.pyplot as plt
# from matplotlib.gridspec import GridSpec
# import numpy as np

# fig = plt.figure(figsize=(10, 8))
# gs = GridSpec(2, 2, figure=fig)

# ######
# cmap1 = plt.cm.viridis
# cmap1 = plt.cm.seismic 
# n_levels=30
# ######

# ######
# vmax_shared = np.max([np.max(profile_array_c_to_g), np.max(profile_array_g_to_c)])
# norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
# ######

# # First subplot: Entrainment
# ########################################
# ax1 = fig.add_subplot(gs[0, 0])
# contour1 = ax1.contourf(profile_array_c_to_g.T, cmap=cmap1,levels=n_levels)
# # contour1 = ax1.contourf(profile_array_e_g.T, cmap=cmap1, norm=norm_shared, levels=n_levels)
# cbar1=fig.colorbar(contour1, ax=ax1)
# Nz = len(data['zh'])
# ax1.set_yticks(np.arange(Nz))
# new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
# ax1.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
# ax1.set_ylabel('z (km)');ax1.set_xlabel('t (timesteps)')
# ax1.set_title('Cloudy to General')

# # Second subplot: Detrainment
# ########################################
# ax2 = fig.add_subplot(gs[0, 1])
# contour2 = ax2.contourf(profile_array_g_to_c.T, cmap=cmap1, levels=n_levels)
# # contour2 = ax2.contourf(profile_array_e_c.T, cmap=cmap1, norm=norm_shared, levels=n_levels)
# cbar2 = fig.colorbar(contour2, ax=ax2)
# ax2.set_yticks(np.arange(Nz))
# new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
# ax2.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
# ax2.set_ylabel('z (km)');ax2.set_xlabel('t (timesteps)')
# ax2.set_title('General to Cloudy')

# ###################### FIXING Y TICKS
# Nz = len(data['zh'])
# step = 4  # change to 2, 5, etc. depending on how spaced you want them
# ytick_pos = np.arange(0, Nz, step)
# ytick_labels = np.round(data['zf'].values[ytick_pos], 2)
# for axis in [ax1,ax2]:
#     axis.set_yticks(ytick_pos)
#     axis.set_yticklabels(ytick_labels, fontsize=8, rotation=0)

# ###################### SCIENTIFIC NOTATION
# apply_scientific_notation_colorbar([cbar1,cbar2])

In [ ]:
##################################################################
#TESTING

In [ ]:
# # Assuming profile_array_e_g is already defined
# Nt = profile_array_e.shape[0]  # Total number of time steps
# Nz = profile_array_e.shape[1]
# zhs = data['zh']

# # Calculate the number of rows and columns needed for the subplots
# cols = 4  # You can adjust this to change the number of columns
# rows = int(np.ceil(Nt / 10 / cols))  # Calculate rows dynamically based on Nt (max 16 plots)

# # Create a figure with a grid of subplots, adjust the figure size for better spacing
# fig, axes = plt.subplots(rows, cols, figsize=(12, 3 * rows))
# axes = axes.flatten()  # Flatten axes to make indexing easier

# # Loop through time steps, creating a plot for each
# for i, t in enumerate(np.arange(0, Nt, 10)):  # Use np.arange with step size 10
#     plot_data = profile_array_net[t]
    
#     # Select the corresponding axis
#     ax = axes[i]
    
#     # Plot the data for the current time step
#     ax.plot(plot_data, zhs, color='black')  # Adjust to match your data's structure
#     ax.axvline(0,linestyle='dashed',color='k')
    
#     # Set title for the subplot
#     ax.set_title(f"Time Step {t}")

#     apply_scientific_notation([ax])

# # Remove any unused axes (if there are fewer plots than grid spaces)
# for j in range(i + 1, len(axes)):
#     axes[j].axis('off')

# # Add global labels and title
# plt.xlabel('X-axis label')  # Replace with actual x-axis label
# plt.ylabel('Z (km)')  # Replace with your y-axis label

# # Adjust layout for better spacing
# plt.tight_layout()
# plt.show()


In [ ]:
# # Assuming profile_array_e_g is already defined
# Nt = profile_array_e.shape[0]  # Total number of time steps
# Nz = profile_array_e.shape[1]
# zhs = data['zh']

# # Calculate the number of rows and columns needed for the subplots
# cols = 4  # You can adjust this to change the number of columns
# rows = int(np.ceil(Nt / 10 / cols))  # Calculate rows dynamically based on Nt (max 16 plots)

# # Create a figure with a grid of subplots, adjust the figure size for better spacing
# fig, axes = plt.subplots(rows, cols, figsize=(12, 3 * rows))
# axes = axes.flatten()  # Flatten axes to make indexing easier

# # Loop through time steps, creating a plot for each
# for i, t in enumerate(np.arange(55,67, 1)):  # Use np.arange with step size 10
#     plot_data = profile_array_net[t]
    
#     # Select the corresponding axis
#     ax = axes[i]
    
#     # Plot the data for the current time step
#     ax.plot(plot_data, zhs, color='black')  # Adjust to match your data's structure
#     ax.axvline(0,linestyle='dashed',color='k')
    
#     # Set title for the subplot
#     ax.set_title(f"Time Step {t}")

#     apply_scientific_notation([ax])

# # Remove any unused axes (if there are fewer plots than grid spaces)
# for j in range(i + 1, len(axes)):
#     axes[j].axis('off')

# # Add global labels and title
# plt.xlabel('X-axis label')  # Replace with actual x-axis label
# plt.ylabel('Z (km)')  # Replace with your y-axis label

# # Adjust layout for better spacing
# plt.tight_layout()
# plt.show()


In [ ]:
# # w_tz=data['winterp'].mean(dim=('xh','yh'))
# # qc_tz=data['qc'].mean(dim=('xh','yh'))
# # # w_tz=data['winterp'].isel(yh=100).mean(dim=('xh'))
# # # qc_tz=data['qc'].isel(yh=100).mean(dim=('xh'))

# # def DdzStretch(f):
# #     import numpy as np
# #     #f must be interpolated to cell centers
# #     dz=np.diff(data['zf'].values)
# #     dz=dz.copy()[np.newaxis, :, np.newaxis, np.newaxis]
    
# #     ddz=np.zeros_like(f)
# #     ddz[:, 1:-1] = (f[:, 2:] - f[:, :-2]) / (2 * dz[:, 1:-1])
# #     ddz[:, 0] = (f[:, 1] - f[:, 0]) / dz[:, 0]  # Forward difference 
# #     ddz[:, -1] = (f[:, -1] - f[:, -2]) / dz[:, -1]  # Backward difference 
# #     return ddz

# # u=data['uinterp'].data
# # dudz=DdzStretch(u)
# # dudz_tz=np.mean(dudz,axis=(2,3))
# # # dudz_tz=np.mean(dudz[:,:,100],axis=(2))

# # w=data['winterp'].data
# # dwdz=DdzStretch(w)
# # dwdz_tz=np.mean(dwdz,axis=(2,3))
# # # dwdz_tz=np.mean(dwdz[:,:,100],axis=(2))

# fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# # Plot the first contour plot for w_tz
# ax1 = axs[0, 0]  # First subplot (top-left)
# c1 = ax1.contourf(w_tz.T,levels=20)  # Transpose if necessary
# fig.colorbar(c1, ax=ax1)
# ax1.set_title('w')

# # Plot the second contour plot for du/dz
# ax2 = axs[0, 1]  # Second subplot (top-right)
# c2 = ax2.contourf(dudz_tz.T,levels=20)  # Transpose if necessary
# fig.colorbar(c2, ax=ax2)
# ax2.set_title('du/dz')

# # Plot the third contour plot for dw/dz
# ax3 = axs[1, 0]  # Third subplot (bottom-left)
# c3 = ax3.contourf(dwdz_tz.T,cmap='RdBu',levels=20,vmax=0.004)  # Transpose if necessary
# fig.colorbar(c3, ax=ax3)
# ax3.set_title('dw/dz')

# # Plot the third contour plot for dw/dz
# ax4 = axs[1, 1]  # Third subplot (bottom-left)
# c4 = ax4.contourf(qc_tz.T,cmap='RdBu',levels=20)  # Transpose if necessary
# fig.colorbar(c4, ax=ax4)
# ax4.set_title('qc')